# Setup

In [1]:
#https://www.kaggle.com/piotrstanczyk/gfootball-template-bot  G-FOOTBALL TEMPLATE BOT
# Install:
# Kaggle environments.
!git clone https://github.com/Kaggle/kaggle-environments.git
!cd kaggle-environments && pip install .

# GFootball environment.
!apt-get update -y
!apt-get install -y libsdl2-gfx-dev libsdl2-ttf-dev

# Make sure that the Branch in git clone and in wget call matches !!
!git clone -b v2.8 https://github.com/google-research/football.git
!mkdir -p football/third_party/gfootball_engine/lib

!wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.8.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so
!cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install .

Cloning into 'kaggle-environments'...
remote: Enumerating objects: 4092, done.
remote: Counting objects: 100% (872/872), done.
remote: Compressing objects: 100% (228/228), done.
remote: Total 4092 (delta 704), reused 755 (delta 644), pack-reused 3220
Receiving objects: 100% (4092/4092), 11.36 MiB | 11.31 MiB/s, done.
Resolving deltas: 100% (2463/2463), done.
Processing /content/kaggle-environments
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for kaggle-environments: filename=kaggle_environments-1.9.5-py3-none-any.whl size=1809949 sha256=33239c020526f770633e45ac8f837fb9f3f83db41bdc35f03033e48fd7576d1d
  Stored in directory: /r

# All the imports

In [38]:
from gfootball.env.football_env import FootballEnv
from kaggle_environments import make
from gfootball.env.config import Config
from gfootball.env.football_env import FootballEnv

#import dqn libraries
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
import numpy as np
import pandas as pd
import itertools
import random
from collections import deque

#import env 
import gym
import gfootball 

#env_name = "GFootballBase-v0"
#print(env_name)

# Deep Q Network Model

In [39]:
class DQN(nn.Module):
  def __init__(self,env):
    super(DQN,self).__init__()
    input_dims = int(np.prod(env.observation_space.shape)) #neurons input layer = number of observations
    self.net =nn.Sequential(nn.Linear(input_dims,130), 
                            nn.Tanh(),
                            nn.Linear(130,env.action_space.n)) # neurons outpul layer = number of 
                            
  def forward(self,x):
    return self.net(x) #use the dqnetwork

  def act(self,obs): 
    obs_t =torch.as_tensor(obs,dtype=torch.float32)
    q_values=self(obs_t.unsqueeze(0)) # make tensor a batch dimension
    max_q_index = torch.argmax(q_values,dim=1)[0]   # taking action with highest q value
    action = max_q_index.detach().item # making tensor to integer which represents action 
    print (action)
    return action 



# Initialize variables

In [4]:
BUFFERSIZE = 50000
REWBUFFERSIZE = 100
MINREPLAYSIZE= 3000 # Episode is 3000 steps
GAMMA = 0.04
EPSILON =0.03

# Agent 

In [31]:
class Agent() : 
  def __init__(self,gamma,epsilon):
    self.gamma =gamma
    self.epsilon =epsilon
    self.ReplayBuffer = deque(maxlen=BUFFERSIZE) #Store experiences 
    self.RewBuffer = deque(maxlen=REWBUFFERSIZE) #Store rewards
    
    self.online_net = DQN(env)   
    self.target_net = DQN(env)  

    self.target_net.load_state_dict(self.online_net.state_dict())

  def transition(self,obs,new_obs,action,reward ,done): # should be tuple ? 
    self.obs =obs
    self.action=action
    self.reward =reward 
    self.done=done
    #self.info =info
    self.new_obs =new_obs
    TransitionTuple= (obs,new_obs,action,reward,done)
    #print("class",TransitionTuple)
    return TransitionTuple



# Create Environment -- Main 


In [47]:
#if __name__ =='main':


env = gym.make("GFootball-11_vs_11_kaggle-simple115v2-v0") #List with the 115 states 
eps_rew =0.0
agent=Agent(GAMMA,EPSILON) #Create object
obs = env.reset()

for i in range(3001): #EPISODE IS 3000 STEPS
  action =env.action_space.sample() #random action 
  new_obs ,reward,done ,info = env.step(action) 
  
  transition = agent.transition(obs,new_obs,action,reward ,done) #obs ,action ,reward , done ,info , new_obs  PROSEKSE TO MALLON LATHOS
  
  agent.ReplayBuffer.append(transition)
  #print("gemisma", agent.ReplayBuffer )
  obs=new_obs
  eps_rew = +reward
  print("|-------|",info ,"for i:",i,"|-------|")
  if(done):
    break
print(obs,done ,info)
  #if(i==3):
   # break

|-------| {'score_reward': 0} for i: 0 |-------|
|-------| {'score_reward': 0} for i: 1 |-------|
|-------| {'score_reward': 0} for i: 2 |-------|
|-------| {'score_reward': 0} for i: 3 |-------|
|-------| {'score_reward': 0} for i: 4 |-------|
|-------| {'score_reward': 0} for i: 5 |-------|
|-------| {'score_reward': 0} for i: 6 |-------|
|-------| {'score_reward': 0} for i: 7 |-------|
|-------| {'score_reward': 0} for i: 8 |-------|
|-------| {'score_reward': 0} for i: 9 |-------|
|-------| {'score_reward': 0} for i: 10 |-------|
|-------| {'score_reward': 0} for i: 11 |-------|
|-------| {'score_reward': 0} for i: 12 |-------|
|-------| {'score_reward': 0} for i: 13 |-------|
|-------| {'score_reward': 0} for i: 14 |-------|
|-------| {'score_reward': 0} for i: 15 |-------|
|-------| {'score_reward': 0} for i: 16 |-------|
|-------| {'score_reward': 0} for i: 17 |-------|
|-------| {'score_reward': 0} for i: 18 |-------|
|-------| {'score_reward': 0} for i: 19 |-------|
|-------| 

In [ ]:
env = make("football", debug=True, configuration={"save_video": False, "scenario_name": "11_vs_11_kaggle", "running_in_notebook": True})

Staring a new environment 1eea7a5c-7ecf-4e15-b672-84c6c4e89b55: with scenario: 11_vs_11_kaggle


In [ ]:
ena =env.reset() 
#dyo=simple_env.reset()

#print(ena.shape)
print(ena[0])#olo to environment
#pprint.pprint(ena[0])


Resetting environment cc712d27-0946-4f08-b0ae-bad76c935e40: with scenario: 11_vs_11_kaggle
{'action': [], 'reward': 0, 'info': {}, 'observation': {'remainingOverageTime': 60, 'step': 0, 'controlled_players': 1, 'players_raw': [{'left_team': [[-1.0110293626785278, -0.0], [-0.4266543984413147, -0.19894461333751678], [-0.5055146813392639, -0.06459399312734604], [-0.5055146813392639, 0.06459297984838486], [-0.4266543984413147, 0.19894461333751678], [-0.18624374270439148, -0.10739918798208237], [-0.270525187253952, -0.0], [-0.18624374270439148, 0.10739918798208237], [-0.010110294446349144, -0.21961550414562225], [-0.05055147036910057, -0.0], [-0.010110294446349144, 0.21961753070354462]], 'right_team_direction': [[-0.0, 0.0], [-0.0, 0.0], [-0.0, 0.0], [-0.0, 0.0], [-0.0, 0.0], [-0.0, 0.0], [-0.0, 0.0], [-0.0, 0.0], [-0.0, 0.0], [-0.0, 0.0], [-0.0, 0.0]], 'right_team_roles': [0, 2, 1, 1, 3, 5, 5, 5, 6, 9, 7], 'right_team_tired_factor': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 

In [ ]:
%%writefile submission.py
from kaggle_environments.envs.football.helpers import *

## @human_readable_agent wrapper modifies raw observations 
# provided by the environment:
# https://github.com/google-research/football/blob/master/gfootball/doc/observation.md#raw-observations
# into a form easier to work with by humans.
# Following modifications are applied:
# - Action, PlayerRole and GameMode enums are introduced.
# - 'sticky_actions' are turned into a set of active actions (Action enum)
#    see usage example below.
# - 'game_mode' is turned into GameMode enum.
# - 'designated' field is removed, as it always equals to 'active'
#    when a single player is controlled on the team.
# - 'left_team_roles'/'right_team_roles' are turned into PlayerRole enums.
# - Action enum is to be returned by the agent function.
@human_readable_agent
def agent(obs):
    # Make sure player is running.
    if Action.Sprint not in obs['sticky_actions']:
        return Action.Sprint
    # We always control left team (observations and actions
    # are mirrored appropriately by the environment).
    controlled_player_pos = obs['left_team'][obs['active']]
    # Does the player we control have the ball?
    if obs['ball_owned_player'] == obs['active'] and obs['ball_owned_team'] == 0:  #EXEI PAIKTIS TIN MPALA KAI EINAI STIN ARISTERI OMADA
        # Shot if we are 'close' to the goal (based on 'x' coordinate).
        if controlled_player_pos[0] > 0.5:
            return Action.Shot
        # Run towards the goal otherwise.
        return Action.Right
    else:
        # Run towards the ball.
        if obs['ball'][0] > controlled_player_pos[0] + 0.05:
            return Action.Right
        if obs['ball'][0] < controlled_player_pos[0] - 0.05:
            return Action.Left
        if obs['ball'][1] > controlled_player_pos[1] + 0.05:
            return Action.Bottom
        if obs['ball'][1] < controlled_player_pos[1] - 0.05:
            return Action.Top
        # Try to take over the ball if close to the ball.
        return Action.Slide

Overwriting submission.py


In [ ]:
# Set up the Environment.
from kaggle_environments import make
env = make("football", configuration={"save_video": True, "scenario_name": "11_vs_11_kaggle", "running_in_notebook": True})
output = env.run(["/kaggle/working/submission.py", "do_nothing"])[-1]
print('Left player: reward = %s, status = %s, info = %s' % (output[0]['reward'], output[0]['status'], output[0]['info']))
print('Right player: reward = %s, status = %s, info = %s' % (output[1]['reward'], output[1]['status'], output[1]['info']))
env.render(mode="human", width=800, height=600)

Left player: reward = None, status = ERROR, info = {}
Right player: reward = 100, status = DONE, info = {'debug_info': 'Opponent forfeited. You win.'}
